In [1]:
def send_message(message):
    recipient = '9365370533' + "@mms.att.net"
    email = "johnodonnell123@gmail.com"
    pw = "vcqe kxvm ruaw djhz"
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(email, pw) 
    server.sendmail(email, recipient, message)
 

In [2]:
# import dependencies
import selenium
import pandas as pd
import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import smtplib
import time

# Log start of scrape
dt = datetime.datetime.today().strftime("%Y-%m-%d %I:%M:%S %p")
print(f'\nStarting Apple Jacks Scrape: {dt}')

# create empty dataframe to store scraped data from all the stores
df = pd.DataFrame()

# define max attempts when program fails (per store)
max_attempts = 2

# define counter for failed attempts
attempt = 1


# loop until the number of failed attempts reaches the max
while attempt <= max_attempts:

    # try becuase sometimes the chrome webdriver randomly fails
    try:

        # define url to start scrape
        url = 'https://applejacksliquor.com/shop?show-search=true&order=price+desc&type=Spirits&subtype=whiskey&style=tiles'   

        # create a dataframe to store the data from this store
        df_store = pd.DataFrame()

        # Set up Selenium WebDriver, run headless so no window appears
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        driver = webdriver.Chrome(options=chrome_options)

        # Navigate to the URL
        driver.get(url)

        # -------------------------- Age Verification -------------------------------

        # Age button 
        age_button_xpath = "//button[@class='age-check-yes']"
        age_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, age_button_xpath)))

        # Click the Change button to open the store selection modal
        age_button.click()
        
        n_pages = 6
        
        for page in range(n_pages):
            
            # -------------------------- PAGE 1 -------------------------------

            # Scrape the names of the bottles from the page 
            bottle_xpath = "//div[@class='ch-product-name']"
            bottles = driver.find_elements(By.XPATH, bottle_xpath)           
            bottle_list = [bottle.text for bottle in bottles]

            # Scrape the prices from the page 
            price_xpath = "//span[@class='ch-single-product-price'] | //div[@class='price-range']"
            prices = driver.find_elements(By.XPATH, price_xpath)
            price_list = [price.text.split(' ')[0].replace('$','').replace(',','') for price in prices]

            # add the data to the store dataframe
            df_store['bottles'] = bottle_list
            df_store['price'] = price_list
            df_store['ts'] = dt

            # append the data from the store dataframe to the master dataframe
            df = pd.concat([df,df_store], ignore_index=True)

            # move to next page
            next_button_xpath = f"//a[@class='button ch-btn nav-button'][last()]"
            next_button = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, next_button_xpath)))

            # Click the next page button
            next_button.click()


        # print out that this attempt was successfull
        print('- Success')
        
        break


    # if the program fails
    except Exception as e:

        # if this is the final attempt print the final failure message 
        if attempt >= max_attempts:
            print('- Final Failure')

        # if this isn't the final attempt print the a failure message and try again
        else:
            print(f'- Failed Attempt #{attempt}')

            # increment attempt to mark this failure
            attempt += 1
            
# Close the browser
driver.quit()

# drop any bad data points
df = df[df['price'] != '']

# get the price
df['price'] = df['price'].astype(float)
              
# read in records (local vs cron hack)
try:
    records = pd.read_csv('Python/web_scraping/Zipps/records/aj_records.csv')
    
except:
    records = pd.read_csv('records/aj_records.csv')

# get new records
new_bottles = [i for i in df['bottles'].unique() if i not in records['bottles'].unique()]
  
# subset dataframe 
new = df[df['bottles'].isin(new_bottles)]

# append new records
records = pd.concat([records, new], ignore_index=True)

# write out records (local vs cron hack)
try:
    records.sort_values(by='ts',ascending=False).to_csv('Python/web_scraping/Zipps/records/aj_records.csv',index=False)
    
except:
    records.sort_values(by='ts',ascending=False).to_csv('records/aj_records.csv',index=False)
    
if len(new_bottles) > 0:
    
    string = ''
    
    # get dataframe of new bottles
    temp = df[df['bottles'].isin(new_bottles)]
    
    # for each new bottle, add name and price to string
    for a,b in zip(temp['bottles'].unique(),temp['price'].unique()):
        string += f'- {a}: {b} \n'
    
    # send text message alert
    try:
        send_message('\nNew Addition to Apple Jacks Records:' + f'\n{string}')
    except:
        print('- Bad Character in string')

    
    
# Log end of scrape
dt = datetime.datetime.today().strftime("%Y-%m-%d %I:%M:%S %p")
print(f'Finished Apple Jacks Scrape: {dt} \n\n ------------------------- \n')


Starting Apple Jacks Scrape: 2023-12-14 09:50:20 AM
- Success
- Jeffersons Tropic Age Hum Bbn 750ml: 109.99 

Finished Apple Jacks Scrape: 2023-12-14 09:50:46 AM 



In [4]:
print(f'Finished Apple Jacks Scrape: {dt} \n\n ------------------------- \n')

Finished Apple Jacks Scrape: 2023-12-14 09:50:46 AM 

 ------------------------- 

